# Домашнее задание 3

Третье домашнее задание посвящено достаточно простой, но, надеюсь, интересной задаче, в которой потребуется творчески применить методы сэмплирования. Как и раньше, в качестве решения **ожидается ссылка на jupyter-ноутбук на вашем github (или публичный, или с доступом для snikolenko); ссылку обязательно нужно прислать в виде сданного домашнего задания на портале Академии**. Как всегда, любые комментарии, новые идеи и рассуждения на тему категорически приветствуются.

В этом небольшом домашнем задании мы **попробуем улучшить метод Шерлока Холмса**. Как известно, в рассказе *The Adventure of the Dancing Men* великий сыщик расшифровал загадочные письмена, которые выглядели как пляшущие человечки.

Пользовался он для этого так называемым частотным методом: смотрел, какие буквы чаще встречаются в зашифрованных текстах, и пытался подставить буквы в соответствии с частотной таблицей: E — самая частая и так далее.

В этом задании мы будем разрабатывать более современный и продвинутый вариант такого частотного метода. В качестве корпусов текстов для подсчётов частот можете взять что угодно, но для удобства вот вам “Война и мир” по-русски и по-английски:

https://www.dropbox.com/s/k23enjvr3fb40o5/corpora.zip

## Задание 1

Реализуйте базовый частотный метод по Шерлоку Холмсу:
* подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
* возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
* расшифруйте их таким частотным методом.

#### Импорт библиотек

In [1]:
import os
import re
import random
from collections import Counter, defaultdict
from copy import copy

import numpy as np
import pandas as pd
from nltk import everygrams
from nltk.tokenize import RegexpTokenizer
from tqdm.notebook import tqdm

In [2]:
np.random.seed(4)

#### Загрузка данных

In [3]:
if not os.path.exists("/content/corpora.zip"):
    !wget -q https://www.dropbox.com/s/k23enjvr3fb40o5/corpora.zip
    !unzip -oq corpora.zip

In [4]:
FILES = ["AnnaKarenina.txt", "WarAndPeace.txt"]

In [5]:
corpus = []
for filename in FILES:
    with open(filename, "r") as fin:
        corpus += fin.readlines()
corpus = " ".join(corpus)

#### Расшифровка

In [6]:
ALPHABET = " абвгдежзийклмнопрстуфхцчшщъыьэюя"

In [7]:
def tokenize(text, alphabet=ALPHABET, tokenizer=RegexpTokenizer(r"\w+")):
    text = text.lower()
    # Filter characters not in alphabet:
    text = "".join([c for c in text if c in alphabet])
    return " ".join(tokenizer.tokenize(text))


def get_ngram_freqs(text, n_gram=1):
    if n_gram > 1:
        text = [
            "".join(ngram) for ngram in everygrams(text, min_len=n_gram, max_len=n_gram)
        ]
    freqs = {
        k: v / len(text)
        for k, v in Counter(text).items()
        if v > 0  # remove zeros, because why do we need them?
    }
    return freqs


def generate_mapping(freqs):
    original = list(freqs.keys())
    replacements = np.random.choice(original, replace=False, size=len(freqs))
    mapping = {
        original_char: replacement_char
        for original_char, replacement_char in zip(original, replacements)
    }
    return mapping


def apply_mapping(text, mapping):
    return "".join([mapping.get(c, "ь") for c in text])


def get_reverse_mapping(corpus_freqs, text_freqs):
    """Нахождение ближайшего по частотности символа."""
    corpus_freqs_sorted = sorted(corpus_freqs.items(), key=lambda x: x[1], reverse=True)
    text_freqs_sorted = sorted(text_freqs.items(), key=lambda x: x[1], reverse=True)

    reverse_mapping = {}
    for text_char, text_freq in text_freqs_sorted:
        min_diff = 1.0  # maximum possible frequency
        best_char = None
        for corpus_char, corpus_freq in corpus_freqs_sorted:
            diff = abs(corpus_freq - text_freq)
            if diff < min_diff:
                best_char = corpus_char
                min_diff = diff

        reverse_mapping[text_char] = best_char
        corpus_freqs_sorted = [
            (char, freq) for char, freq in corpus_freqs_sorted if char != best_char
        ]

    return reverse_mapping


def character_accuracy(text1, text2):
    assert len(text1) == len(text2)
    matching_chars = sum((c1 == c2) for c1, c2 in zip(text1, text2))
    return matching_chars / len(text1)

In [8]:
tokenized_corpus = tokenize(corpus)
corpus_freqs = get_ngram_freqs(tokenized_corpus, n_gram=1)
mapping = generate_mapping(corpus_freqs)

In [9]:
text = """
    Наша работа во тьме —\n
    Мы делаем, что умеем,\n
    Мы отдаем, что имеем,\n
    Наша работа – во тьме.\n
    Сомнения стали страстью,\n
    А страсть стала судьбой.\n
    Все остальное — искусство\n
    В безумии быть собой.\n
    \n
    Хочется закрыть глаза. Это нормально. Цветной калейдоскоп, блестки, искрящийся звездный вихрь – красиво, но я знаю, что стоит за этой красотой.\n
    Глубина. Ее называют «дип», но мне кажется, что по-русски слово звучит правильнее. Заменяет красивый ярлычок предупреждением. Глубина! Здесь водятся акулы и спруты. Здесь тихо – и давит, давит, давит бесконечное пространство, которого на самом деле нет.\n
    В общем-то она добрая, глубина. По-своему, конечно. Она принимает любого. Чтобы нырнуть, нужно не много сил. Чтобы достичь дна и вернуться – куда больше. В первую очередь надо помнить – глубина мертва без нас. Надо и верить в нее, и не верить. Иначе настанет день, когда не удастся вынырнуть.
"""

In [10]:
tokenized_text = tokenize(" ".join(text.split("\n")))
encoded_text = apply_mapping(tokenized_text, mapping)
text_freqs = get_ngram_freqs(encoded_text)
tokenized_text

'наша работа во тьме мы делаем что умеем мы отдаем что имеем наша работа во тьме сомнения стали страстью а страсть стала судьбой все остальное искусство в безумии быть собой хочется закрыть глаза это нормально цветной калейдоскоп блестки искрящийся звездный вихрь красиво но я знаю что стоит за этой красотой глубина ее называют дип но мне кажется что порусски слово звучит правильнее заменяет красивый ярлычок предупреждением глубина здесь водятся акулы и спруты здесь тихо и давит давит давит бесконечное пространство которого на самом деле нет в общемто она добрая глубина посвоему конечно она принимает любого чтобы нырнуть нужно не много сил чтобы достичь дна и вернуться куда больше в первую очередь надо помнить глубина мертва без нас надо и верить в нее и не верить иначе настанет день когда не удастся вынырнуть'

In [11]:
encoded_text

'вщфщудщсшмщупшум лзулеуызнщзлукмшуилззлулеушмыщзлукмшучлззлувщфщудщсшмщупшум лзуршлвзвчэурмщнчурмдщрм оущурмдщрм урмщнщуриы сшбупрзушрмщн вшзучрьиррмпшупусзаилччусем уршсшбухшкзмрэуащьдем уюнщащуямшувшдлщн вшуъпзмвшбуьщнзбышрьшгуснзрмьчучрьдэтчбрэуапзаывебупчхд уьдщрчпшувшуэуавщоукмшурмшчмуащуямшбуьдщршмшбуюнисчвщуззувщаепщомуычгувшулвзуьщжзмрэукмшугшдиррьчурншпшуапикчмугдщпчн вззуащлзвэзмуьдщрчпебуэднекшьугдзыигдзжызвчзлуюнисчвщуаызр упшыэмрэущьинеучургдимеуаызр умчхшучуыщпчмуыщпчмуыщпчмусзрьшвзквшзугдшрмдщврмпшуьшмшдшюшувщурщлшлуызнзувзмупушстзлмшушвщуышсдщэуюнисчвщугшрпшзлиуьшвзквшушвщугдчвчлщзмуносшюшукмшсеуведвим увижвшувзулвшюшурчнукмшсеуышрмчк уывщучупздвим рэуьиыщусшн фзупугздпиоушкздзы увщышугшлвчм уюнисчвщулздмпщусзаувщрувщышучупздчм упувззучувзупздчм учвщкзувщрмщвзмуызв уьшюыщувзуиыщрмрэупеведвим '

In [12]:
reverse_mapping = get_reverse_mapping(corpus_freqs, text_freqs)
decoded_text = apply_mapping(encoded_text, reverse_mapping)
decoded_text

'тнэн рняеин ве имда дг капнад ыие удаад дг еикнад ыие лдаад тнэн рняеин ве имда седтатлб синпл сирнсимх н сирнсим синпн сукмяеж вса есинпмтеа лсьуссиве в яазудлл ягим сеяеж цеыаисб зньргим йпнзн фие терднпмте юваитеж ьнпажкесьеч япасиьл лсьрбължсб звазктгж влцрм ьрнслве те б зтнх ыие сиели зн фиеж ьрнсеиеж йпуялтн аа тнзгвнхи клч те дта ьнщаисб ыие черуссьл спеве звуыли чрнвлпмтаа зндатбаи ьрнслвгж брпгыеь чракучращкатлад йпуялтн зкасм векбисб ньупг л счруиг зкасм илце л кнвли кнвли кнвли яасьетаытеа чресирнтсиве ьеиерейе тн сндед капа таи в еяъадие етн кеярнб йпуялтн чесвеаду ьетаыте етн чрлтлднаи пхяейе ыиеяг тгртуим тущте та дтейе слп ыиеяг кесилым ктн л вартуимсб ьукн яепмэа в чарвух еыаракм тнке чедтлим йпуялтн даривн яаз тнс тнке л варлим в таа л та варлим лтныа тнсинтаи катм ьейкн та укнсисб вгтгртуим'

In [13]:
character_accuracy(tokenized_text, decoded_text)

0.326007326007326

Текст нечитаем, посимвольная точность расшифровки оставляет желать много лучшего.

## Задание 2

Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:
* подсчитайте частоты *биграмм* (т.е. пар последовательных букв) по корпусам;
* проведите тестирование аналогично п. 1, но при помощи биграмм. В качестве естественной метрики качества можно взять долю правильно расшифрованных букв или, если хочется совсем математически изощриться, расстояние между двумя перестановками, правильной и полученной из модели; но, честно говоря, в этом задании следить за численными метриками не так уж обязательно, будет и глазами всё видно.

In [14]:
corpus_freqs_bigram = get_ngram_freqs(tokenized_corpus, n_gram=2)
text_freqs_bigram = get_ngram_freqs(encoded_text, n_gram=2)

corpus_freqs_sorted = sorted(
    corpus_freqs_bigram.items(), key=lambda x: x[1], reverse=True
)
text_freqs_sorted = sorted(text_freqs_bigram.items(), key=lambda x: x[1], reverse=True)

In [15]:
def get_reverse_mapping_ngram(corpus_freqs, text_freqs, n_gram=1):
    corpus_freqs_sorted = sorted(corpus_freqs.items(), key=lambda x: x[1], reverse=True)
    text_freqs_sorted = sorted(text_freqs.items(), key=lambda x: x[1], reverse=True)

    # Iterate over n-grams starting from the most frequent. On each iteration
    # we take into account already decoded symbols:
    reverse_mapping = {}
    for i, (text_ngram, text_freq) in enumerate(text_freqs_sorted):
        filtered_freqs = copy(corpus_freqs_sorted)

        for j in range(n_gram):
            if text_ngram[j] in reverse_mapping:
                filtered_freqs = [
                    (ngram, freq)
                    for ngram, freq in filtered_freqs
                    if ngram[j] == reverse_mapping[text_ngram[j]]
                ]

        min_diff = 1.0  # maximum possible frequency
        best_ngram = None
        for ngram, freq in filtered_freqs:
            diff = abs(freq - text_freq)
            if diff < min_diff:
                best_ngram = ngram
                min_diff = diff

        for j in range(n_gram):
            if text_ngram[j] not in reverse_mapping:
                reverse_mapping[text_ngram[j]] = best_ngram[j]

    return reverse_mapping

In [16]:
reverse_mapping_bigram = get_reverse_mapping_ngram(
    corpus_freqs_bigram, text_freqs_bigram, n_gram=2
)
decoded_text = apply_mapping(encoded_text, reverse_mapping_bigram)
decoded_text

'тоео  ово о со  ела лл оасоал д о клаал лл о ооал д о илаал тоео  ово о со  ела аолтатия а оси а  оа ею о а  оа е а осо акоевол саа оа осетоа иаикаа со с вакклии вл е аовол сода ая кои л е чсоко ж о то лосето хса тол иосалооаиок всаа ии иаи ятилая ксакотлл сис е и оаисо то я ктою д о а ои  ко ж ол и оао ол чсквито аа токлсою  оик то лта иоша ая д о ко кааии асосо кскди  к осисетаа колатяа  и оаислл я слдои к аокк ашоатиал чсквито коаае сооя ая оиксл и ак к л коаае  исо и ооси  ооси  ооси  вааиотадтоа к оа  ота со ио о очо то аолол оаса та  с овтал о ото оов оя чсквито коасоалк иотадто ото к итилоа  сювочо д овл тл тк е ткшто та лточо аис д овл ооа иде ото и са тк еая икоо восееа с ка скю ода аое тооо колти е чсквито ла  со вак тоа тооо и са и е с таа и та са и е итода тоа ота  оате иочоо та кооа ая слтл тк е'

In [17]:
character_accuracy(tokenized_text, decoded_text)

0.3125763125763126

Качество слегка ухудшилось. Объяснить это можно тем, что биграмм намного больше, чем униграмм, попасть по частоте в правильные — сложнее.

## Задание 3

Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:
* предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
* реализуйте и протестируйте его, убедитесь, что результаты улучшились.

Текст, разбитый на биграммы — это, по сути, марковская цепь. Частота биграмм — вероятность перехода между состояниями цепи.

Для обучения перестановки будем считать вероятность порождения именно такого текста как произведение вероятностей всех биграмм, в него входящих.

Всего перестановок очень много, поэтому будем использовать жадный алгоритм, основанный на идее MCMC-сэмплирования.

Алгоритм:
1. Инициализируем перестановки, восстанавливаем текст и вычисляем логарифм правдоподобия $p_{current}$.
2. Меняем местами пару символов для перестановки.
3. Восстанавливаем текст с новой перестановкой и вычисляем $p_{proposed}$.
4. Принимаем новую перестановку с "вероятностью" $\displaystyle p_{accept} = \frac{p_{proposed}}{p_{current}}$.
5. Возвращаемся к пункту 2 и повторяем цикл.

Мы можем уйти немного не туда и застрять не в том максимуме, поэтому на каждой итерации алгоритма будем делать много попыток и брать в качестве окончательного результата лучшую из них.

In [18]:
def get_ngram_freqs_smoothed(text, n_gram=2):
    vocab_size = len(set(text)) ** n_gram
    if n_gram > 1:
        text = [
            "".join(ngram) for ngram in everygrams(text, min_len=n_gram, max_len=n_gram)
        ]
    freqs = {
        k: (v + 1) / (len(text) + vocab_size)  # сглаживаем, чтобы не было нулей
        for k, v in Counter(text).items()
    }
    return freqs


def get_text_proba(text, mapping, freqs, n_gram=2, alphabet=ALPHABET):
    decoded_text = apply_mapping(text, mapping)
    log_proba = 0.0
    for i in range(len(decoded_text) - n_gram):
        ngram = decoded_text[i : i + n_gram]
        ngram_proba = freqs.get(
            ngram, 1 / (len(text) + len(alphabet) ** n_gram)
        )  # сглаживаем, чтобы не было нулей
        log_proba += np.log(ngram_proba)
    return log_proba


def get_reverse_mapping_mcmc(
    encoded_text,
    alphabet_encoded,
    alphabet_corpus,
    freqs_corpus,
    n_iters=10000,
    n_trials=10,
    n_gram=2,
):
    accept_count = 0
    best_reverse_mapping = None
    all_mappings = []
    best_log_likelihood = -np.inf

    for trial in tqdm(range(n_trials), leave=False, position=0, total=n_trials):
        alphabet_encoded = list(alphabet_encoded)
        alphabet_iter = list(alphabet_corpus)
        reverse_mapping = {
            k: v
            for k, v in zip(alphabet_encoded, alphabet_iter[: len(alphabet_encoded)])
        }
        log_proba_current = get_text_proba(
            encoded_text, reverse_mapping, freqs_corpus, n_gram=n_gram
        )

        for i in range(n_iters):
            alphabet_proposal = alphabet_iter[:]
            idx1, idx2 = np.random.choice(len(alphabet_proposal), replace=False, size=2)
            alphabet_proposal[idx1], alphabet_proposal[idx2] = (
                alphabet_proposal[idx2],
                alphabet_proposal[idx1],
            )
            reverse_mapping_proposal = {
                k: v
                for k, v in zip(
                    alphabet_encoded, alphabet_proposal[: len(alphabet_encoded)]
                )
            }
            log_proba_proposal = get_text_proba(
                encoded_text, reverse_mapping_proposal, freqs_corpus, n_gram=n_gram
            )

            p_accept = np.exp(log_proba_proposal - log_proba_current)

            if p_accept > np.random.rand():
                accept_count += 1
                alphabet_iter = alphabet_proposal
                log_proba_current = log_proba_proposal
                reverse_mapping = reverse_mapping_proposal

        if log_proba_current > best_log_likelihood:
            best_log_likelihood = log_proba_current
            best_reverse_mapping = reverse_mapping

        all_mappings.append(reverse_mapping)

    print(f"Best likelihood: {best_log_likelihood}")
    print(f"Accept ratio: {accept_count / (n_iters * n_trials)}")
    return best_reverse_mapping

In [19]:
freqs_corpus = get_ngram_freqs_smoothed(tokenized_corpus, n_gram=2)

In [20]:
best_reverse_mapping = get_reverse_mapping_mcmc(
    encoded_text,
    alphabet_encoded=ALPHABET,
    alphabet_corpus=ALPHABET,
    freqs_corpus=freqs_corpus,
)

Best likelihood: -4490.216747523113
Accept ratio: 0.01544


In [21]:
decoded_text = apply_mapping(encoded_text, best_reverse_mapping)
decoded_text

'наша работа во тьме мы делаем что умеем мы отдаем что имеем наша работа во тьме сомнения стали страстью а страсть стала судьбой все остальное искусство в безумии быть собой хочется закрыть глаза это нормально фветной калейдоскоп блестки искрящийся звездный вихрь красиво но я знаю что стоит за этой красотой глубина ее называют дип но мне кажется что порусски слово звучит правильнее заменяет красивый ярлычок предупреждением глубина здесь водятся акулы и спруты здесь тихо и давит давит давит бесконечное пространство которого на самом деле нет в общемто она добрая глубина посвоему конечно она принимает любого чтобы нырнуть нужно не много сил чтобы достичь дна и вернуться куда больше в первую очередь надо помнить глубина мертва без нас надо и верить в нее и не верить иначе настанет день когда не удастся вынырнуть'

In [22]:
character_accuracy(tokenized_text, decoded_text)

0.9987789987789988

Получилось!

## Задание 4

Расшифруйте сообщение:
`←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏`

Или это (они одинаковые, второй вариант просто на случай проблем с юникодом):
`დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ`


In [23]:
message = "←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏"

In [24]:
message_freqs = get_ngram_freqs(message, n_gram=1)

In [25]:
corpus_freqs_sorted = sorted(corpus_freqs.items(), key=lambda x: x[1], reverse=True)
message_freqs_sorted = sorted(message_freqs.items(), key=lambda x: x[1], reverse=True)

alphabet_corpus = "".join([c for c, _ in corpus_freqs_sorted])
alphabet_message = "".join([c for c, _ in message_freqs_sorted])
alphabet_corpus, alphabet_message

(' оеанитслвркдмупяьгыбзчжйшхюэцщфъ', '↹←⇛↟⇒↝⇴↨⇠⇯↷⇌⇊⇞⇈⇷↤↳↾↙⇙↲↞⇆⇰⇸↘⇏')

In [26]:
freqs_corpus = get_ngram_freqs_smoothed(tokenized_corpus, n_gram=2)

In [27]:
best_reverse_mapping = get_reverse_mapping_mcmc(
    message,
    alphabet_encoded=alphabet_message,
    alphabet_corpus=alphabet_corpus,
    freqs_corpus=freqs_corpus,
    n_iters=10000,
    n_trials=50,
)

Best likelihood: -1231.2547551943435
Accept ratio: 0.042954


In [28]:
encoded_message = apply_mapping(message, best_reverse_mapping)
encoded_message

'если вы вимите нордальный или почти нордальный текст у этого соожшения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный жалл за послемнее четвертое замание курса ботя конечно я ничего не ожешах'

Тут тоже получилось. :) Посмотрим на посимвольную ошибку:

In [29]:
original_message = "если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаю"

In [30]:
character_accuracy(original_message, encoded_message)

0.9347826086956522